<a href="https://colab.research.google.com/github/akarajic/CUS615/blob/main/ProblemSet8/FINAL_Problem_set_08_Exam_Project_CUS615_(1)_(1)_(2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook is part the of **Dr. Christoforos Christoforou's** course materials. You may not, nor may you knowingly allow others to reproduce or distribute lecture notes, course materials or any of their derivatives without the instructor's express written consent.

# Exam Project
**Professor**: Dr. Christoforos Christoforou

The objective of this project is to reproduce the data analysis method presented in the paper:

* **"Mining Big Data to Extract Patterns and Predict Real-Life Outcomes"**
by Michal Kosinski, Yilun Wang, Himabindu Lakkaraju, and Jure Leskovec.


You must implement the analysis using python code. This notebook provides you with the starting code to load the data and guides you through the requirement and each sub-task of the analysis. To complete this project you are expected to:

1. Carefully read the paper.
2. Refer the class lecture where this paper was discussed in detail.
3. Research various python libraries and utility functions needed to implement your analysis.
4. Complete all the challenges outlined in this notebook.



## Student Information 
Complete your information in the form provided below. This is visible in colab. 


In [ ]:
#@title Strudent Information 
student_name = "Adna Karajic" #@param {type:"string"}
course_code = "CUS 615" #@param {type:"string"}


## Starter code - Configuring the analysis environment
The code cells below provide you a starting code to setup the environment for you to complete this project's tasks.

In particular, the code downloads the three dataset associated with this project and stores them in the folder `/content/sample_data/`. The data files from the paper comprise the `likes.csv` file, the `users.csv` file and the `users-likes.csv`; description of each data file is availabile in the paper.




In [70]:
#
# Run this sell to download the data for your analysis. 
#
!wget -O /content/sample_data/sample_dataset.zip https://osf.io/9m87k/download
!unzip /content/sample_data/sample_dataset.zip -d /content/sample_data/


--2023-05-03 00:28:22--  https://osf.io/9m87k/download
Resolving osf.io (osf.io)... 35.190.84.173
Connecting to osf.io (osf.io)|35.190.84.173|:443... connected.
HTTP request sent, awaiting response... 302 FOUND
Location: https://files.osf.io/v1/resources/fuqjg/providers/osfstorage/5e946850f1353503a3d52f24?action=download&direct&version=1 [following]
--2023-05-03 00:28:22--  https://files.osf.io/v1/resources/fuqjg/providers/osfstorage/5e946850f1353503a3d52f24?action=download&direct&version=1
Resolving files.osf.io (files.osf.io)... 35.186.214.196
Connecting to files.osf.io (files.osf.io)|35.186.214.196|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 314322145 (300M) [application/octet-stream]
Saving to: ‘/content/sample_data/sample_dataset.zip’

/content/sample_dat 100%[===================>] 299.76M  69.1MB/s    in 4.2s    

2023-05-03 00:28:27 (70.9 MB/s) - ‘/content/sample_data/sample_dataset.zip’ saved [314322145/314322145]

Archive:  /content/sample_data/sa

In [71]:
#
# Install additional package not available in colab.
#
!pip install factor_analyzer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#
# You will likely need the following python packages when implementing this project.
# Add any additional package as you deem necessary 

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import svds, eigs
from factor_analyzer import Rotator

# Additional package you want to use:


## Challenge 1 : Load the data into dataframes
In this challenge you are expected to load the data from the raw files into the following variables:
-  `users` : should reference a dataframe with the data in the `users.csv` file
-  `likes` : should reference a dataframe with the data in the `likes.csv` file
-  `ul` : should reference a dataframe with the data in `users.csv` file

Then print the top 10 rows of reach dataframe.

In [100]:
import pandas as pd

users = pd.read_csv("./sample_data/users.csv")
likes = pd.read_csv("./sample_data/likes.csv")
ul = pd.read_csv("./sample_data/users-likes.csv")


In [101]:
#
# Answer summary cell: Print the top 10 row from each datafame. 
# 

print("Users: ", users.head())
print("Likes: ", likes.head())
print("User-Likes: ", ul.head())

Users:                               userid  gender  age  political   ope   con   ext  \
0  54f34605aebd63f7680e37ffd299af79       0   33        0.0  1.26  1.65  1.17   
1  86399f8c44ba54224b2e60177ca89fa9       1   35        0.0  1.07  0.17 -0.14   
2  84fab50f3c60d1fdc83aa91b5e584a78       1   36        0.0  0.89  1.28  0.86   
3  f3b8fdaccce12ef6352bfad4d6052fe9       0   39        NaN  0.33 -1.01 -0.33   
4  8b06ea5e9cb87c61da387995450607f7       0   31        NaN  0.15  0.47  1.17   

    agr   neu  
0 -1.76  0.61  
1  1.49  0.30  
2  1.07  0.99  
3 -0.68  0.92  
4 -1.01 -0.32  
Likes:                               likeid  \
0  3c1636c878e6eb2acfd00c6b61086e38   
1  feca46ddb8ef04f86172ace0cb7e004c   
2  b65f46d64c688fe98bdbcf93a76a71fc   
3  9c5c8bb82d2cd46fbd7582f944fe370e   
4  2d82fa84ad79b085dc516dde154327a2   

                                                name  
0                               REIGN by Paul Gibson  
1                   Cupcake Wishes & Birthday Dreams  
2

## Challenge 2: Create the sparse user-likes matrix 
Create a **sparse matrix** that stores the data for the user-likes associations as described in the paper. The name of the user-likes matrix must be `M`, to keep the naming convention used in the paper. It is important that to store the data in a sparse matrix; you might need to research the topic of using sparse matrices in python (i.e. look at the `scipy.sparse` package and the `csr_matrix` class). *Hint: For this challenge,  you might find  the `merge` method from `pandas` library useful*. 

Once you create the matrix print its shape on the answer summary cell.

In [103]:
from scipy.sparse import csr_matrix

#Merge the 'ul' and 'users' dataframes on the 'userid' column and get the 'index' column
rows = pd.merge(left=ul, right=users.reset_index(), on='userid', how="left")['index']

#Merge the 'ul' and 'likes' dataframes on the 'likeid' column and get the 'index' column
columns = pd.merge(left=ul, right=likes.reset_index(), on="likeid", how="left")['index']

M = csr_matrix((np.ones((len(ul))), (rows,columns)))

In [104]:
print("Shape of the Raw Matrix M:", M.shape)


Shape of the Raw Matrix M: (110728, 1580284)


## Challege 3: Trim the sparse user-likes matrix M.
The paper calls for the matrix `M` to be trimmed (i.e. to remove users which did not like a minimum number of posts and posts that did not receive a minimum number of likes). In this challenge, you are expected to implement the trimming preprocessing step as described in the paper. The resulting matrix must be a sparse matrix. *Hint: you might want to research to index slice sparse matrices in python*

In [105]:
y=users.political

#Remove rows and columns where the sum of values is less than 50 and 150 respectively
while True:
    i = M.shape[0] + M.shape[1]
    y = y[np.array(np.sum(M, axis=1) >= 50).flatten()]
    M = M[np.array(np.sum(M, axis=1) >= 50).flatten(), :]
    M = M[:,np.array(np.sum(M, axis=0) >= 150).flatten()]
    if i == (M.shape[0] + M.shape[1]):
        break

y= pd.DataFrame(y).reset_index(drop=True)

In [106]:
print("Shape of the Trimmed Matrix M:", M.shape)

Shape of the Trimmed Matrix M: (19742, 8523)


## Challenge 4:  Split the sparse matrix into a training and testing matrices
To build a prediction model the paper calls for separating the data matrix (i.e. the user-like matrix) into a training and testing matrices. As part of this challenge, you need to create a training matrix `M_train` that includes the preferences of 80% of the users, and a testing matrix `M_test` that include the preferences of the remaining 20% of the users in the dataset. Both matrices `M_train` and `M_test` must be stored as sparse matrices. User will be randomly assigned to the training or testing set.

Print the shapes of the `M_train` and `M_test` matrices.




In [107]:
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.model_selection import train_test_split

#Split the row indices into training and test sets
M_train, M_test, y_train, y_test = train_test_split(M, y, train_size=0.8, random_state=42)

In [108]:
#
# Answer summary cell: Print the shapes of M_train, M_test and M matrices.  
# 

print(M_train.shape)
print(M_test.shape)
print(y_train.shape)
print(y_test.shape)

(15793, 8523)
(3949, 8523)
(15793, 1)
(3949, 1)


## Challege 5 : Reduce the dimensionality of the M_train matrix using SVD

In this challenge, you must use singular value decomposition (SVD) to reduce the dimensionality of the training matrix `M_train`. Remember that `M_train` is encoded as a sparse matrix so you would need to use an appropriate implementation of the SVD for sparse matrices. Your output should be the matrices `U`, `S` and `Vt` that correspond to the left singular vectors, the singular values, and the right singular vectors, respectively. You must specify a parameter `k` that defines the number of reduced dimensions you want to keep.

Print the shapes of the `U`, `S` and `Vt` matrices and the value you selected for parameter `k`.



In [109]:
from scipy.sparse.linalg import svds
import numpy as np

# Specify the number of dimensions to keep
k = 200

# Compute the SVD of M_train
U, S, Vt = svds(M_train, k=k, random_state=12)

In [110]:
print(U.shape)
print(S.shape)
print(Vt.shape)
print(k)

(15793, 200)
(200,)
(200, 8523)
200


## Challege 6 : Use the varimax algorithm to rotate the SVD dimensions. 
Use the varimax algorithm to rotate the SVD dimensions as described in the paper. A good python library that implements varimax algorithm is the `factor_analyzer` and its `Rotator` class (already installed in this notebook for your in the started cell above); i.e. using `rotator = Rotator(method='varimax')`. Store the varimax SVD into the matrices `U_rot` and `V_rot` respectively. *Hint: you can apply varimax of `Vt` to get `V_rot` and then multiply `M` by  `Vt_rot` to obtain the `U_rot` (as done in the paper); you might want to research hot to user the `Rotator` class*. 

Print the shapes of the matrices `U_rot` and `V_rot`.


In [111]:
from factor_analyzer.rotator import Rotator

# Create a Rotator object with varimax method
rotator = Rotator(method='varimax')

# Fit the Rotator object on Vt matrix to obtain V_rot
V_rot = rotator.fit_transform(Vt.T)

# Multiply M by the rotated Vt matrix to obtain U_rot
U_rot = M.dot(V_rot)

In [112]:
print(V_rot.shape)
print(U_rot.shape)

(8523, 200)
(19742, 200)


## Challenge 7 : Train two predictive model to predict the political  affilication of users.
Build a predictive model that predicts the political affiliation of the users given the user's post preferences (as those are captured in the paper's dataset). To train your model you can use data from matrix `M_train` only. You are expected to use two different predictive models of your choice (i.e. Knn, SVM, Logistic Regression or some other model of your choice).

As part of this challenge you are expected to:
- Apply any necessary preprocessing to handle missing values.
- The models must be applied on the reduced-dimensional feature generated by your analysis above (i.e. use the singular rotated singular dimensions)
- Use cross-validation for model selection (i.e. to identify optimal parameters for your model, or any meta-parameters)
- Report training and cross-validation performance of the two models as well as the optimal parameters for your model identified by cross validation. 


**FIRST MODEL TRAINING: Use the cells below to train the predictive model using the first model you selected.**

In [139]:
#
# Use this cell to train the first model you selected for challenge 7. 
# Add additional cells as needed below this cell.
#
import pandas as pd

#Calculate the  product of M_train and the transpose of Vt and assign it to X_train
X_train=M_train.dot(Vt.T)

#Convert X_train into a DataFrame object and add y_train as a column
X_train=pd.DataFrame(X_train)
X_train['y']=y_train

#Remove rows with null values in the y column
X_train=X_train[X_train['y'].isnull()==False]
X_train['y'].isnull()

#Assign the y_train column to the y_train variable and drop the y column from X_train
y_train=X_train['y']
X_train=X_train.drop('y', axis=1)

**FIRST MODEL ACCURACY REPORT: Use the cell below to report on the accuracy of your first model**


In [132]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

# Define parameter grid for SVM
param_grid = {'C': [0.01, 0.1, 1, 10],
              'gamma': [0.01, 0.1, 1, 10]}

# Initialize SVM model
svm_model = SVC(kernel='rbf', random_state=42)

# Perform grid search with cross-validation to find optimal hyperparameters
svm_grid_search = GridSearchCV(svm_model, param_grid, cv=5)
svm_grid_search.fit(X_train, y_train)

# Get the best hyperparameters and the corresponding mean test score
best_params = svm_grid_search.best_params_
best_score = svm_grid_search.best_score_

# Train the SVM model with the best hyperparameters on the full training set
svm_model = SVC(kernel='rbf', C=best_params['C'], gamma=best_params['gamma'], random_state=42)

print("Best parameters:", best_params)
print("Best cross-validation score:", best_score)

Best parameters: {'C': 1, 'gamma': 0.01}
Best cross-validation score: 0.7017763327355524


**SECOND MODEL TRAINING: Use the cells below to train the predictive model using the second model you selected.**

In [134]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

# Define the range of hyperparameters for KNN
param_grid = {
    'n_neighbors': range(1,100),
    'weights': ['uniform', 'distance'],
    'p': [1, 2]
}

# Create a KNN model
knn = KNeighborsClassifier()

# Perform grid search cross-validation to select the optimal hyperparameters
grid_search = GridSearchCV(knn, param_grid, cv=5, n_jobs=-1, scoring='accuracy')
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=KNeighborsClassifier(), n_jobs=-1,
             param_grid={'n_neighbors': range(1, 100), 'p': [1, 2],
                         'weights': ['uniform', 'distance']},
             scoring='accuracy')

**SECOND MODEL ACCURACY REPORT: Use the cell below to report on the accuracy of your second model**

In [135]:
# Print the best hyperparameters and their associated performance
print('Best hyperparameters:', grid_search.best_params_)
print('Best cross-validation score:', grid_search.best_score_)


Best hyperparameters: {'n_neighbors': 46, 'p': 2, 'weights': 'distance'}
Best cross-validation score: 0.7027113981755531


## Challenge 8 Apply your model in the independent test set.
User the cells below to apply your two predictive models on the test dataset `M_test` and report testing accuracy for each model.

As part of this challenge you are expected to:
- Project each observation in the test dataset `M_test` onto the reduced-dimensional features space.
- Apply each on of your model to make predict each user's political preference (for users in the testing set only)
- Report testing accuracy on each of your models.
- Make an assessment as to which model is better and whether either of your models is overfitted or under-fitted.

In [136]:
#Calculate the  product of M_test and the transpose of Vt and assign it to X_test
X_test=M_test.dot(Vt.T)

#Convert X_test into a DataFrame object and add y_test as a column
X_test=pd.DataFrame(X_test)
X_test['y']=y_test
X_test.head()

#Remove rows with null values in the y column
X_test=X_test[X_test['y'].isnull()==False]
X_test['y'].isnull().value_counts

#Assign the y_test column to the y_test variable and drop the y column from X_test
y_test=X_test['y']
X_test=X_test.drop('y', axis=1)


In [137]:
svm_model.fit(X_train, y_train)

# Evaluate the SVM model on the test set
svm_test_acc = svm_model.score(X_test, y_test)

print("SVM Test accuracy:", svm_test_acc)

SVM Test accuracy: 0.7692307692307693


In [138]:
knn_model = KNeighborsClassifier(n_neighbors=46, weights='distance', p=2)
knn_model.fit(X_train,y_train)
y_pred = knn_model.predict(X_test)

# Calculate the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)
print("KNN Test accuracy: {:.2f}".format(accuracy))

KNN Test accuracy: 0.77


# END OF EXAM/CODE


*Copyright Statement*: Copyright © 2020 Christoforou. The materials provided by the instructor of this course, including this notebook, are for the use of the students enrolled in the course. Materials are presented in an educational context for personal use and study and should not be shared, distributed, disseminated or sold in print — or digitally — outside the course without permission. You may not, nor may you knowingly allow others to reproduce or distribute lecture notes, course materials  as well as any of their derivatives without the instructor's express written consent.